## Import program dependencies

In [1]:
import cv2 as cv
import numpy as np

## Load the serialized black and white colorizer model and cluster center points

In [2]:
prototxt = '[YOUR WORKSPACE]\\colorization_deploy_v2.prototxt'
model = '[YOUR WORKSPACE]\\colorization_release_v2.caffemodel'
points = '[YOUR WORKSPACE]\\pts_in_hull.npy]'
net = cv.dnn.readNetFromCaffe(prototxt, model)
pts = np.load(points)

## Add the cluster centers as 1x1 convolutions to the model

In [3]:
class8 = net.getLayerId('class8_ab')
conv8 = net.getLayerId('conv8_313_rh')
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype('float32')]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype = 'float32')]

## Load (and display) the input image

In [4]:
filename = '[YOUR WORKSPACE]\\[YOUR IMAGE.JPG]'
image = cv.imread(filename)
cv.imshow('Original', image)

## Scale the pixel intensities to the range [0, 1]

In [5]:
scaled = image.astype('float32') / 255.0

## Convert the image from BGR to Lab

In [6]:
lab = cv.cvtColor(scaled, cv.COLOR_BGR2LAB)

## Resize the image to 224 x 224 (required size for the model)

In [7]:
resized = cv.resize(lab, (224, 224))

## Extract the L channel, then perform mean centering

In [8]:
Lchannel = cv.split(resized)[0]
Lchannel -= 50

## Pass the L channel through the network, which then returns the ab values

In [9]:
net.setInput(cv.dnn.blobFromImage(Lchannel))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

## Resize the ab channel to the size of the original image

In [10]:
ab = cv.resize(ab, (image.shape[1], image.shape[0]))

## Grab the L channel (from the original image, not the resized one)

In [11]:
L = cv.split(lab)[0]

## Concatenate the L channel to the ab channel (this creates the colorized image)

In [12]:
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis = 2)

## Convert the colorized image from Lab back to BGR

In [13]:
colorized = cv.cvtColor(colorized, cv.COLOR_LAB2BGR)

## Clip any values that fall outside the range [0, 1]

In [14]:
colorized = np.clip(colorized, 0, 1)

## Scale the pixel intensities back to the range [0, 255]

In [15]:
colorized = (255 * colorized).astype('uint8')

## Display the result

In [16]:
cv.imshow('Result', colorized)
cv.waitKey(0)

-1